<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/master/landlab_header.png"></a>

# Mapping values between grid elements

Imagine that you're using Landlab to write a model of shallow water flow over terrain. A natural approach is to place your scalar fields, such as water depth, at the nodes. You then place your vector fields, such as water surface slope, flow velocity, and discharge, at the links (or, alternatively, faces). But your velocity depends on both slope and depth, which means you need to know the depth at the links too. How do you do this?

This tutorial introduces *mappers*: grid functions that map quantities defined on one set of elements (such as nodes) onto another set of elements (such as links). As you'll see, there are a variety of mappers available.



## Mapping from nodes to links

For the sake of example, we'll start with a simple 3-row by 4-column raster grid. The grid will contain a scalar field called *some_field*, abbreviated *h*. We'll populate it with some example values, as follows:

In [30]:
from landlab import RasterModelGrid
import numpy as np
mg = RasterModelGrid(3, 4)
h = mg.add_zeros('node', 'some_field')
h[:] = 7 - np.abs(6 - np.arange(12))

For the sake of visualizing values at nodes on our grid, we'll define a handy little function:

In [31]:
def show_node_values(mg, u):
    for r in range(mg.number_of_node_rows - 1, -1, -1):
        for c in range(mg.number_of_node_columns):
            print int(u[c + (mg.number_of_node_columns * r)]),
        print

In [32]:
show_node_values(mg, h)

5 4 3 2
5 6 7 6
1 2 3 4


Let's review the numbering of nodes and links. The lines below will print a list that shows, for each link ID, the IDs of the nodes at the link's tail and head:

In [28]:
for i in range(mg.number_of_links):
    print i, mg.node_at_link_tail[i], mg.node_at_link_head[i]

0 0 1
1 1 2
2 2 3
3 0 4
4 1 5
5 2 6
6 3 7
7 4 5
8 5 6
9 6 7
10 4 8
11 5 9
12 6 10
13 7 11
14 8 9
15 9 10
16 10 11


### Finding the mean value between two nodes on a link

Let's imagine that *h* represents water depth, and that we want to have a *link-based* array, called *h_edge*, that also contains water depth. For each link, we'll simply take the average of the depth at the link's two nodes. To accomplish this, we can use the `map_mean_of_link_nodes_to_link` grid method. At link 8, for example, we'll average the *h* values at nodes 5 and 6, which should give us a depth of (6 + 7) / 2 = 6.5:

In [33]:
h_edge = mg.map_mean_of_link_nodes_to_link('some_field')
for i in range(mg.number_of_links):
    print i, h_edge[i]

0 1.5
1 2.5
2 3.5
3 3.0
4 4.0
5 5.0
6 5.0
7 5.5
8 6.5
9 6.5
10 5.0
11 5.0
12 5.0
13 4.0
14 4.5
15 3.5
16 2.5


### What's in a name?

The mapping function has a long name, which is designed to make it as clear as possible to understand what the function does. All the mappers start with the verb *map*. Then the relationship is given; in this case, we are looking at the *mean*. Then the elements from which a quantity is being mapped: we are taking values from *link nodes*. Finally, the element to which the new values apply: *link*.

### Mapping minimum or maximum values

We can also map the minimum value of *h*:

In [34]:
h_edge = mg.map_min_of_link_nodes_to_link('some_field')
for i in range(mg.number_of_links):
    print i, h_edge[i]

0 1.0
1 2.0
2 3.0
3 1.0
4 2.0
5 3.0
6 4.0
7 5.0
8 6.0
9 6.0
10 5.0
11 4.0
12 3.0
13 2.0
14 4.0
15 3.0
16 2.0


... or the maximum:

In [35]:
h_edge = mg.map_max_of_link_nodes_to_link('some_field')
for i in range(mg.number_of_links):
    print i, h_edge[i]

0 2.0
1 3.0
2 4.0
3 5.0
4 6.0
5 7.0
6 6.0
7 6.0
8 7.0
9 7.0
10 5.0
11 6.0
12 7.0
13 6.0
14 5.0
15 4.0
16 3.0


### Upwind and downwind

Numerical schemes often use *upwind differencing* or *downwind differencing*. For example, finite difference schemes for equations that include advection may use "upwind" rather than centered differences, in which a scalar quantity (our *h* for example) is taken from whichever side is upstream in the flow field.

How do we know the flow direction? If the flow is driven by the gradient in some scalar field, such as pressure or elevation, one approach is to look at the values of this scalar on either end of the link: the end with the higher value is upwind, and the end with the lower value is downwind.

Suppose for example that our water flow is driven by the water-surface slope (which is often a good approximation for the *energy slope*, though it omits the kinetic energy). Let's define a bed-surface elevation field *z*:

In [42]:
z = mg.add_zeros('node', 'topographic__elevation', noclobber=False)
z[:] = 16 - np.abs(7 - np.arange(12))
show_node_values(mg, z)

15 14 13 12
13 14 15 16
9 10 11 12


The water-surface elevation is then the sum of *h* and *z*:

In [43]:
w = z + h
show_node_values(mg, w)

20 18 16 14
18 20 22 22
10 12 14 16


For every link, we can assign the value of *h* from whichever end of the link has the greater *w*:

In [46]:
h_edge = mg.map_value_at_max_node_to_link(w, h)
for i in range(mg.number_of_links):
    print i, h_edge[i]

0 2.0
1 3.0
2 4.0
3 5.0
4 6.0
5 7.0
6 6.0
7 6.0
8 7.0
9 6.0
10 5.0
11 6.0
12 7.0
13 6.0
14 5.0
15 4.0
16 3.0


Consider the middle two nodes (5 and 6). Node 6 is higher (22 versus 20). Therefore, the link between them (link 8) should be assigned the value of *h* at node 6. This value happens to be 7.0.

Of course, we could also take the value from the *lower* of the two nodes, which gives link 8 a value of 6.0:

In [47]:
h_edge = mg.map_value_at_min_node_to_link(w, h)
for i in range(mg.number_of_links):
    print i, h_edge[i]

0 1.0
1 2.0
2 3.0
3 1.0
4 2.0
5 3.0
6 4.0
7 5.0
8 6.0
9 6.0
10 5.0
11 4.0
12 3.0
13 2.0
14 4.0
15 3.0
16 2.0


TO DO NEXT: FINISH THIS BIT, DEMONSTRATING THE UPWIND/DOWNWIND STYLE MAPPERS

STUFF STILL TO BE WOVEN IN:

In [6]:
h_edge = mg.map_link_head_node_to_link('some_field')
for i in range(mg.number_of_links):
    print i, h_edge[i]

0 5.0
1 4.0
2 3.0
3 2.0
4 1.0
5 0.0
6 1.0
7 1.0
8 0.0
9 1.0
10 2.0
11 3.0
12 4.0
13 5.0
14 3.0
15 4.0
16 5.0


In [7]:
h_edge = mg.map_link_tail_node_to_link('some_field')
for i in range(mg.number_of_links):
    print i, h_edge[i]

0 6.0
1 5.0
2 4.0
3 6.0
4 5.0
5 4.0
6 3.0
7 2.0
8 1.0
9 0.0
10 2.0
11 1.0
12 0.0
13 1.0
14 2.0
15 3.0
16 4.0


In [52]:
def test_upwind_thingy(mg, link_val, node_val):
    #flowdir = np.where(node_val[mg.node_at_link_tail] > node_val[mg.node_at_link_tail])
    out = mg.map_link_tail_node_to_link(node_val)
    (w, ) = np.where(link_val < 0.0)
    out[w] = node_val[mg.node_at_link_head[w]]
    return out

In [55]:
g = mg.calculate_gradients_at_links(w)
lv = test_upwind_thingy(mg, -g, h)
for ln in range(mg.number_of_links):
    print ln, -g[ln], h[mg.node_at_link_tail[ln]], h[mg.node_at_link_head[ln]], lv[ln]

0 -2.0 1.0 2.0 2.0
1 -2.0 2.0 3.0 3.0
2 -2.0 3.0 4.0 4.0
3 -8.0 1.0 5.0 5.0
4 -8.0 2.0 6.0 6.0
5 -8.0 3.0 7.0 7.0
6 -6.0 4.0 6.0 6.0
7 -2.0 5.0 6.0 6.0
8 -2.0 6.0 7.0 7.0
9 -0.0 7.0 6.0 7.0
10 -2.0 5.0 5.0 5.0
11 2.0 6.0 4.0 6.0
12 6.0 7.0 3.0 7.0
13 8.0 6.0 2.0 6.0
14 2.0 5.0 4.0 5.0
15 2.0 4.0 3.0 4.0
16 2.0 3.0 2.0 3.0
